In [1]:

import pandas as pd

gt_path = "/mnt/adsinglecell/fatemeh/rosmapnetworks/pbmc68k/For Filipi/Data/Raw/pbmc68k/68k_pbmc_barcodes_annotation.tsv"

# Load ground truth annotations
gt_data = pd.read_csv(gt_path, sep="\t")
gt_data = gt_data[["barcodes", "celltype"]]  # Keep only relevant columns



In [ ]:
import scanpy as sc
import scipy.io
import pandas as pd
import scvi
from sklearn.metrics import adjusted_rand_score

# Paths to your files
mtx_file = "/mnt/adsinglecell/fatemeh/rosmapnetworks/pbmc68k/For Filipi/Data/Raw/pbmc68k/pbmc68k_expression_matrix.mtx"
cells_file = "/mnt/adsinglecell/fatemeh/rosmapnetworks/pbmc68k/For Filipi/Data/Raw/pbmc68k/cell barcodes.txt"
genes_file = "/mnt/adsinglecell/fatemeh/rosmapnetworks/pbmc68k/For Filipi/Data/Raw/pbmc68k/genes.txt"

# Load the matrix (cells x genes)
X = scipy.io.mmread(mtx_file).tocsr()  # Transpose to match AnnData format

# Load cell names
cell_names = pd.read_csv(cells_file, header=None, sep="\t")[0].values

# Load gene names
gene_names = pd.read_csv(genes_file, header=None, sep="\t")[0].values

# Create AnnData object
adata = sc.AnnData(X)
adata.obs_names = cell_names  # Assign cell names
adata.var_names = gene_names  # Assign gene names


In [3]:
# Normalize & log transform
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# Filter genes & cells if needed (optional)
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.filter_cells(adata, min_genes=200)


In [4]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.cuda.is_available()

False

In [ ]:
# Assuming you have ground truth labels in 'true_labels.txt'
#/mnt/adsinglecell/fatemeh/rosmapnetworks/pbmc68k/For Filipi/Data/Raw/pbmc68k/68k_pbmc_barcodes_annotation.tsv


# Setup scVI
scvi.model.SCVI.setup_anndata(adata)

# Train scVI model
model = scvi.model.SCVI(adata)
model.train(max_epochs=200)

# Get the learned latent representation (deep features)
adata.obsm["X_scVI"] = model.get_latent_representation()




In [4]:
from sklearn.cluster import KMeans
# Define number of clusters (adjust based on expected cell types)
num_clusters = 11  # Change this based on your dataset

# Perform K-Means clustering on the latent space
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
adata.obs["kmeans_clusters"] = kmeans.fit_predict(adata.obsm["X_scVI"])


In [5]:
import pandas as pd

# Load ground truth labels
true_labels_df = pd.read_csv(
    "/mnt/adsinglecell/fatemeh/rosmapnetworks/pbmc68k/For Filipi/Data/Raw/pbmc68k/68k_pbmc_barcodes_annotation.tsv",
    sep="\t"
)

# Check the first few rows
print(true_labels_df.head())

      TSNE.1     TSNE.2          barcodes                      celltype
0   7.565540   0.441370  AAACATACACCCAA-1              CD8+ Cytotoxic T
1   2.552626 -25.786672  AAACATACCCCTCA-1  CD8+/CD45RA+ Naive Cytotoxic
2  -5.771831  11.830846  AAACATACCGGAGA-1           CD4+/CD45RO+ Memory
3   1.762556  25.979346  AAACATACTAACCG-1                       CD19+ B
4 -16.793856 -16.589970  AAACATACTCTTCA-1               CD4+/CD25 T Reg


In [6]:
# Ensure barcode names match AnnData's obs_names
true_labels_dict = true_labels_df.set_index("barcodes")["celltype"].to_dict()

# Assign ground truth labels to the AnnData object
adata.obs["true_labels"] = adata.obs_names.map(true_labels_dict)

# Remove any unmatched cells (if necessary)
adata = adata[adata.obs["true_labels"].notna(), :]

In [7]:
from sklearn.metrics import adjusted_rand_score

# Ensure clustering labels are in a proper format
predicted_labels = adata.obs["kmeans_clusters"].astype(str)  # Or use "gmm_clusters" if using GMM
true_labels = adata.obs["true_labels"].astype(str)

# Compute ARI score
ari_score = adjusted_rand_score(true_labels, predicted_labels)
print(f"Adjusted Rand Index (ARI): {ari_score:.4f}")


Adjusted Rand Index (ARI): 0.2041
